# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [383]:
# Import required libraries and dependencies
import pandas as pd
import hvplot 
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [384]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [385]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [386]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [387]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [388]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [389]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
# YOUR CODE HERE!
k = list(range(1, 11))

In [390]:
# Create an empy list to store the inertia values
# YOUR CODE HERE!
inertia = []

In [391]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
# YOUR CODE HERE!
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)

In [392]:
# Create a dictionary with the data to plot the Elbow curve
# YOUR CODE HERE!
elbow_data = {
    'k' : k,
    'inertia' : inertia
}

# Create a DataFrame with the data to plot the Elbow curve
# YOUR CODE HERE!
df_elbow_data = pd.DataFrame(elbow_data)

In [393]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# YOUR CODE HERE!
df_elbow_data.hvplot(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** 4 is the best value for k

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [394]:
# Initialize the K-Means model using the best value for k
# YOUR CODE HERE!
model = KMeans(n_clusters=4)

In [395]:
# Fit the K-Means model using the scaled data
# YOUR CODE HERE!
model.fit(df_market_data_scaled)

KMeans(n_clusters=4)

In [396]:
# Predict the clusters to group the cryptocurrencies using the scaled data
# YOUR CODE HERE!
clusters = model.predict(df_market_data_scaled)

# View the resulting array of cluster values.
# YOUR CODE HERE!
print(clusters)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 3 1 0 0 2
 0 0 0 0]


In [397]:
# Create a copy of the DataFrame
# YOUR CODE HERE!
df_market_data_scaled_clusters = df_market_data_scaled.copy()

In [398]:
# Add a new column to the DataFrame with the predicted clusters
# YOUR CODE HERE!
df_market_data_scaled_clusters['Cluster'] = clusters 

# Display sample data
# YOUR CODE HERE!
df_market_data_scaled_clusters.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,1
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,1
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,1


In [399]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
# YOUR CODE HERE!
df_market_data_scaled_clusters.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='Cluster',
    hover_cols='coin_id',
    title='Scatter Plot by Cryptocurrency - k=4'
)

:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [400]:
# Create a PCA model instance and set `n_components=3`.
# YOUR CODE HERE!
pca = PCA(n_components=3)

In [401]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
# YOUR CODE HERE!
pca_data = pca.fit_transform(df_market_data_scaled_clusters)

# View the first five rows of the DataFrame. 
# YOUR CODE HERE!
pca_data[:5]

array([[-0.13836417, -1.02022407,  0.62338208],
       [-0.05986698, -0.62367162,  1.09276322],
       [-0.62559828,  0.08496615, -0.64815172],
       [-0.65985101,  0.12262437, -0.48303724],
       [-0.24145509, -2.35559929,  1.91883243]])

In [402]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
# YOUR CODE HERE!
pca.explained_variance_ratio_

array([0.37938672, 0.32923147, 0.18455968])

#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** Approx. 89% 

In [403]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you

# Creating a DataFrame with the PCA data
# YOUR CODE HERE!
crypto_pca_data = pd.DataFrame(pca_data, columns=['PC1', 'PC2', 'PC3'])

# Copy the crypto names from the original data
# YOUR CODE HERE!
crypto_pca_data['coin_id'] = df_market_data_scaled.index

# Set the coinid column as index
# YOUR CODE HERE!
crypto_pca_data = crypto_pca_data.set_index('coin_id')

# Display sample data
# YOUR CODE HERE!
crypto_pca_data.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.138364,-1.020224,0.623382
ethereum,-0.059867,-0.623672,1.092763
tether,-0.625598,0.084966,-0.648152
ripple,-0.659851,0.122624,-0.483037
bitcoin-cash,-0.241455,-2.355599,1.918832


---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [404]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
# YOUR CODE HERE!
k2 = list(range(1,11))

In [405]:
# Create an empy list to store the inertia values
# YOUR CODE HERE!
inertia2 = []

In [406]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
# YOUR CODE HERE!
for i in k2:
    model2 = KMeans(n_clusters=i, random_state=1)
    model2.fit(crypto_pca_data)
    inertia2.append(model2.inertia_)

In [407]:
# Create a dictionary with the data to plot the Elbow curve
# YOUR CODE HERE!
elbow2 = {
    'k2': k2,
    'inertia2': inertia2
 }

# Create a DataFrame with the data to plot the Elbow curve
# YOUR CODE HERE!
elbow_data2 = pd.DataFrame(elbow2)

In [408]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# YOUR CODE HERE!
elbow_data2.hvplot(
    x='k2',
    y='inertia2',
    title='PCA Elbow Curve',
    xticks=k2
)

:Curve   [k2]   (inertia2)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [409]:
# Initialize the K-Means model using the best value for k
# YOUR CODE HERE!
model2 = KMeans(n_clusters=4)

In [410]:
# Fit the K-Means model using the PCA data
# YOUR CODE HERE!
model2.fit(crypto_pca_data)

KMeans(n_clusters=4)

In [411]:
# Predict the clusters to group the cryptocurrencies using the PCA data
# YOUR CODE HERE!
clusters2 = model2.predict(crypto_pca_data)

# View the resulting array of cluster values.
# YOUR CODE HERE!
clusters2

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [412]:
# Create a copy of the DataFrame with the PCA data
# YOUR CODE HERE!
crypto_pca_data_clusters = crypto_pca_data.copy()

# Add a new column to the DataFrame with the predicted clusters
# YOUR CODE HERE!
crypto_pca_data_clusters['Clusters'] = clusters2

# Display sample data
# YOUR CODE HERE!
crypto_pca_data_clusters

,PC1,PC2,PC3,Clusters
coin_id,,,,
bitcoin,-0.138364,-1.020224,0.623382,2
ethereum,-0.059867,-0.623672,1.092763,2
tether,-0.625598,0.084966,-0.648152,0
ripple,-0.659851,0.122624,-0.483037,0
bitcoin-cash,-0.241455,-2.355599,1.918832,2
binancecoin,0.097551,-1.527764,0.878127,2
chainlink,0.120633,-0.722223,2.858698,2
cardano,0.153565,-0.864766,1.543101,2
litecoin,-0.271292,-0.642959,0.795960,2


In [413]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
# YOUR CODE HERE!
crypto_pca_data_clusters.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='Clusters',
    hover_cols='coin_id',
    title='Scatter Plot by Cryptocurrency PCA - k=4'
)

:NdOverlay   [Clusters]
   :Scatter   [PC1]   (PC2,coin_id)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [416]:
# Composite plot to contrast the Elbow curves
# YOUR CODE HERE!
elbow_curve = df_elbow_data.hvplot(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xticks=k
)

elbow_curve_PCA = elbow_data2.hvplot(
    x='k2',
    y='inertia2',
    title='PCA Elbow Curve',
    xticks=k2
)

composite_elbow = elbow_curve + elbow_curve_PCA

composite_elbow

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k2]   (inertia2)

In [414]:
# Compoosite plot to contrast the clusters
# YOUR CODE HERE!
scatter_plot = df_market_data_scaled_clusters.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='Cluster',
    hover_cols='coin_id',
    title='Scatter Plot by Cryptocurrency - k=4'
)

scatter_plot_PCA = crypto_pca_data_clusters.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='Clusters',
    hover_cols='coin_id',
    title='Scatter Plot by Cryptocurrency PCA - k=4'
)

composite_plot = scatter_plot + scatter_plot_PCA

composite_plot

:Layout
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Clusters]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** After using PCA for dimensionality reduction, the inertia of clusters is lower